# Pytorch 数据集
在深度学习任务中，数据加载和处理是至关重要的。

Pytorch提供了一个强大的数据加载和处理的工具，`torch.utils.data`模块。这个模块提供了一个抽象的接口来处理数据集和数据加载器。其包含的工具主要有：
- `torch.utils.data.Dataset`：一个抽象类，表示一个数据集，需要自定义并实现`__len__`和`__getitem__`（按索引获取样本）方法；
- `torch.utils.data.DataLoader`：封装`Dataset`的迭代器，提供批处理、打乱、并行加载等功能，便于数据输入模型训练；
- `torch.utils.data.TensorDataset`：基于Tensor的简单数据集，适用于小型数据集，适合处理数据-标签对，直接支持批处理和迭代；
- `torchvision.datasets.ImageFolder`：从文件夹加载图像数据，每个子文件夹代表一个类别，适用于图像分类任务；
- `torchvision.transforms`：用于图像预处理的工具，提供了多种常用的图像变换操作，如裁剪、缩放、翻转、归一化等；
- `torchvision.datasets`：提供了多种常用的数据集，如MNIST、CIFAR-10等，方便快速加载和使用；
- `torch.utils.data.random_split`：用于将数据集随机划分为训练集和验证集，便于模型评估；
- `torch.utils.data.ConcatDataset`：用于将多个数据集合并为一个数据集，便于处理多个数据源；
- `torch.utils.data.Subset`：用于从数据集中获取子集，便于处理大规模数据集；

## 1. Pytorch 内置数据集
pytorch通过`torchvision.datasets`模块提供了许多常用的数据集，例如：
- MNIST：手写数字图像数据集，用于图像分类任务；
- CIFAR-10：小型图像数据集，包含10个类别的60000张32x32彩色图像，用于图像分类任务；
- CIFAR-100：小型图像数据集，包含100个类别的60000张32x32彩色图像，用于图像分类任务；
- COCO：大型图像数据集，包含80个类别的330000张图像和2.5M个目标实例，用于目标检测和分割任务；
- ImageNet：大型图像数据集，包含1000个类别的1400000张图像，用于图像分类和物体检测任务；
- STL-10：小型图像数据集，包含10个类别的10000张32x32彩色图像，用于图像分类任务；
- Cityscapes：大型图像数据集，包含19个类别的5000张高分辨率街景图像，用于语义分割任务；
- SQUAD：用于机器阅读理解任务的数据集，包含100000个问题和答案对；
- IMDB：用于情感分析任务的数据集，包含50000条影评文本和对应的情感标签；
- AG News：用于文本分类任务的数据集，包含120000条新闻文本和对应的类别标签；

## 2. torchvision和torchtext
- `torchvision`：一个图像库，提供了图片数据处理相关的API和数据接口，包括数据集加载函数和常用的图像变换操作；
- `torchtext`：一个文本库，提供了文本数据处理相关的API和数据接口，包括数据预处理和数据加载的方式；

## 3. `torch.tuils.data.Dataset`
`Dataset`是一个抽象类，用于表示一个数据集。

用户需要自定义一个类，继承`Dataset`类，并重写以下方法：
- `__init__`：初始化数据集，通常用于加载数据和标签；
- `__len__`：返回数据集的大小；
- `__getitem__`：根据索引获取数据和标签，通常用于返回一个样本的数据和标签；

In [1]:
import torch
from torch.utils.data import Dataset

# self defined dataset
class MyDataset(Dataset):
    def __init__(self, data, labels):
        # Initialize the dataset with data and labels
        self.data = data
        self.labels = labels

    def __len__(self):
        # Return the number of samples in the dataset
        return len(self.data)

    def __getitem__(self, idx):
        # Get the data and label for a given index
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label
    
# generate the example data
data = torch.randn(100, 5)             # 100 samples, 5 features for each
labels = torch.randint(0, 2, (100,))   # binary labels (0 or 1)

# instantiate the dataset
dataset = MyDataset(data, labels)

# test the dataset
print(f"Number of samples: {len(dataset)}")
print(f"First sample: {dataset[0]}")

Number of samples: 100
First sample: (tensor([-0.9653, -0.4638,  1.0233,  0.3844,  1.1839]), tensor(0))


## 4. `torch.utils.data.DataLoader`
`DataLoader`是pytorch提供的数据加载器，用于批量加载数据集。其提供了以下功能：
- 批量加载数据：可以指定每个批次的大小`batch_size`；
- 打乱数据：可以在每个`epoch`开始时打乱数据`shuffle=True`，增加模型的泛化能力；
- 多线程加载：可以使用多个子进程并行加载数据，`num_workers`参数可以指定使用的子进程数量，提高数据加载效率；
- 迭代访问：可以使用`for`循环迭代访问数据集，返回每个批次的数据和标签；
- 自动填充：可以自动填充不完整的批次，确保每个批次的大小一致；

In [4]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# self defined dataset
class MyDataset(Dataset):
    def __init__(self, data, labels):
        # Initialize the dataset with data and labels
        self.data = data
        self.labels = labels

    def __len__(self):
        # Return the number of samples in the dataset
        return len(self.data)

    def __getitem__(self, idx):
        # Get the data and label for a given index
        sample = self.data[idx]
        label = self.labels[idx]
        return sample, label

# generate the example data
data = torch.randn(100, 5)             # 100 samples, 5 features for each
labels = torch.randint(0, 2, (100,))   # binary labels (0 or 1)

# instantiate the dataset
dataset = MyDataset(data, labels)
# create a DataLoader
dataloader = DataLoader(dataset, batch_size=10, shuffle=True, num_workers=0)

# traverse the DataLoader
for batch_idx, (batch_data, batch_labels) in enumerate(dataloader):
    print(f"Batch {batch_idx + 1}:")
    print(f"Data: {batch_data}")
    print(f"Labels: {batch_labels}")
    # break after first batch for demonstration
    if batch_idx == 1:
        break

Batch 1:
Data: tensor([[-1.2374, -0.8510,  0.5050,  1.0553, -0.9911],
        [-0.0390,  0.5259, -1.4702,  0.8122, -0.2769],
        [-0.7355, -0.0571, -0.0048,  0.4285,  0.4695],
        [ 0.0878, -0.2396, -0.5731, -0.0430, -1.0814],
        [-1.0058, -0.6285, -0.2464, -0.5730, -0.8918],
        [ 0.4183,  0.4852, -0.7359, -0.1474, -0.4360],
        [-0.8915, -0.2917, -0.4202, -0.1014, -0.3616],
        [ 1.3316, -0.3271, -2.7973,  1.4087, -0.6980],
        [ 1.0853, -0.1313,  0.0483, -2.2979, -0.5655],
        [ 0.5651,  0.0771, -1.7851, -2.9000, -1.0320]])
Labels: tensor([1, 0, 0, 1, 1, 1, 0, 0, 1, 0])
Batch 2:
Data: tensor([[-0.0587, -1.2635, -0.1506, -1.4105, -0.8655],
        [-1.4113, -0.7016,  1.1347, -0.5281, -1.2413],
        [ 0.6348, -0.0290,  1.0258, -1.0792, -0.4326],
        [ 0.2605, -0.3408, -0.3078,  1.5287, -0.6771],
        [-1.0151,  0.2030,  1.2668, -1.5381, -1.1854],
        [-1.0034,  0.1417, -2.0599, -1.0575, -0.1368],
        [ 2.4683, -1.0232,  1.2880,  0.208

## 5. 使用内置数据集
Pytorch提供了多个常用数据集，存放在`torchvision.datasets`模块中。使用这些数据集非常简单，只需要调用相应的函数即可。

In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# define data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),                  # convert to tensor
    transforms.Normalize((0.5,), (0.5,))    # normalize to mean=0.5, std=0.5
])

# load the train dataset
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)

# load data through DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)

# check a batch of data
data_iter = iter(train_loader)
images, labels = next(data_iter)
print(f"The size of the batch: {images.shape}")
print(f"The size of the labels: {labels}")

The size of the batch: torch.Size([64, 1, 28, 28])
The size of the labels: tensor([9, 8, 0, 3, 3, 8, 0, 7, 8, 3, 6, 3, 8, 3, 7, 3, 8, 8, 7, 4, 2, 3, 1, 0,
        9, 3, 0, 0, 1, 2, 2, 2, 2, 2, 4, 8, 0, 8, 2, 2, 1, 9, 6, 0, 0, 3, 1, 6,
        8, 7, 7, 0, 1, 1, 9, 0, 6, 0, 7, 8, 7, 2, 6, 7])


## 6. `Dataset`和`DataLoader`的自定义使用
将一个CSV文件作为数据源，并通过自定义`Dataset`类和`DataLoader`类来加载数据。以下是一个简单的示例：

In [6]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader

# self defined dataset
class MyDataset(Dataset):
    def __init__(self, csv_file):
        # Initialize the dataset with a CSV file
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        # Return the number of samples in the dataset
        return len(self.data)

    def __getitem__(self, idx):
        # use .iloc to ensure the data is in the correct format
        row = self.data.iloc[idx]
        # separate features and labels
        features = torch.tensor(row.iloc[:-1].to_numpy(), dtype=torch.float32)
        label = torch.tensor(row.iloc[-1], dtype=torch.float32)
        return features, label
    
# instantiate the dataset
dataset = MyDataset('./data/runoob_pytorch_data.csv')
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# traverse the DataLoader
for features, labels in dataloader:
    print(f"Features: {features}")
    print(f"Labels: {labels}")
    # break after first batch for demonstration
    break

Features: tensor([[-1.1000,  0.8000,  1.5000],
        [ 0.5000, -1.2000,  3.3000],
        [ 1.2000,  2.1000, -3.0000],
        [ 0.9000, -0.5000, -1.8000]])
Labels: tensor([1., 1., 1., 0.])
